In [1]:
import numpy as np
import pandas as pd

### 读取数据

In [2]:
def get_train_data():
    data = pd.read_csv("/home/kangaroo/project/LiHongYi/HomeWork1/data/train.csv")
    pm2_5 = data.loc[data["observation"] == "PM2.5"].iloc[:,3:]
    x, y = [], []
    for i in range(15):
        x_ = pm2_5.iloc[:, i:i+9]
        x_.columns = np.array(range(9))
        y_ = pm2_5.iloc[:, 9]
        y_.columns = [0]
        x.append(x_)
        y.append(y_)
    X = np.array(pd.concat(x), int)
    Y = np.array(pd.concat(y), int)
    X = np.c_[np.ones((X.shape[0], 1)), X]
    return X, Y

In [3]:
def get_test_data():
    data = pd.read_csv("/home/kangaroo/project/LiHongYi/HomeWork1/data/test.csv")
    pm2_5 = data.loc[data.iloc[:, 1] == "PM2.5"].iloc[:,2:]
    X = np.array(pm2_5, int)
    X = np.c_[np.ones((X.shape[0], 1)), X]
    return X

### 建立模型
#### 1. Adagrad
#### 2. Mini-Batch
- 数据量太少，所以不需要用Mini-Batch

In [4]:
class LinearRegression():
    def __init__(self, lr = 0.1, epsilon =1e-8 ,epochs = 1000, batch_size = 64):
        self.lr = lr
        self.epsilon = epsilon
        self.epochs = epochs
        self.batch_size = 64
        
    def fetch_batch(self, batch_index, x, y):
        x_batch = x[batch_index * self.batch_size : batch_index * (self.batch_size + 1), :]
        y_batch = y[batch_index * self.batch_size : batch_index * (self.batch_size + 1)]
        return x_batch, y_batch
    
    def output(self, x):
        return np.dot(x, self.w)
        
    
    def fit(self, x, y):
        self.w = np.zeros(x.shape[1])
        all_grad = np.zeros(x.shape[1])
        
        n_batches = int(np.ceil(len(y) / self.batch_size))
        
        for epoch in range(self.epochs):
#             for batch_index in range(n_batches):
#                 x_batch, y_batch = self.fetch_batch(batch_index, x, y)
                
            y_hat = self.output(x)
            error = y - y_hat
            grad = -1 * x.T.dot(error)
            all_grad += grad ** 2
            adagrad = np.sqrt(all_grad.sum())
            self.w -= self.lr * grad / (adagrad + self.epsilon)

    def r2(self, y_true, y_predict):
        return 1 - np.sum((y_true - y_predict) ** 2) / np.sum((y_true - y_true.mean()) ** 2)
    
    def rmse(self, y_true, y_predict):
        return np.sqrt(np.mean((y_true - y_predict) ** 2))

In [5]:
x_train, y_train = get_train_data()
x_test = get_test_data()

In [6]:
LR = LinearRegression()
LR.fit(x_train, y_train)

### 根据模型进行预测

In [7]:
res = LR.output(x_test)

### 查看模型R-square指数

In [8]:
LR.r2(y_train, LR.output(x_train))

0.5622674741023397

In [9]:
LR.rmse(y_train, LR.output(x_train))

10.097933670000668

模型效果不好，待进一步优化

- 交叉验证

### 保存预测结果到指定文件

In [10]:
ans = pd.read_csv("/home/kangaroo/project/LiHongYi/HomeWork1/data/sampleSubmission.csv")

In [11]:
ans["value"] = res.astype(np.int32)

In [12]:
ans.to_csv("/home/kangaroo/project/LiHongYi/HomeWork1/data/sampleSubmission.csv", index=False)